<a href="https://colab.research.google.com/github/Mendietazo/Diplomado-Ciencia-de-Datos-2024/blob/main/Diplomado-Ciencia-de-Datos-2024/Omar-Entregas-/proyecto_mibici_parte_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# YOUR CODE HERE:
import pandas as pd
import plotly.express as px
import math
from google.colab import drive
from datetime import datetime
from datetime import timedelta

drive.mount('/content/drive')
%cd /content/drive/ My \Drive/Diplomado/archive
nomen = pd.read_csv("nomenclature_2024.csv")
df = pd.read_csv("mibici_2014-2024/mibici_2014-2024.csv", parse_dates = ['Trip_start','Trip_end'])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/ My Drive/Diplomado/archive


# Comparación de uso de MiBici con otos medios de transporte


In [ ]:
# Duración del viaje
df['Duration'] = pd.to_timedelta(df['Duration'])

In [ ]:
#Creamos la variable de año
df['Year'] = df['Trip_start'].apply(lambda x: x.year)
# Día de la semana
df['dia_semana'] = df['Trip_start'].apply(lambda x: x.weekday())
#Sábado 5
#Domingo 6

In [ ]:
ajuste = timedelta(
    minutes = 31
)

In [ ]:
#Cargo adicional
df['Cargo_adicional'] = 0

In [ ]:

#Cargo Adicional No Lunes
df['Cargo_adicional'][(df['Duration'] >= timedelta(seconds=1, minutes=30)) & (df['dia_semana'] != 6)] = 28
df['Cargo_adicional'][(df['Duration'] >= timedelta(seconds=1, minutes=60)) & (df['dia_semana'] != 6)] += (df['Duration'][(df['Duration'] >= timedelta(seconds=1, minutes=30)) & (df['dia_semana'] != 6)] - ajuste).dt.total_seconds() // 1800 * 39

#Cargo Adicional No Lunes
df['Cargo_adicional'][(df['Duration'] >= timedelta(seconds=1, minutes=45)) & (df['dia_semana'] == 6)] = 28
df['Cargo_adicional'][(df['Duration'] >= timedelta(seconds=1, minutes=90)) & (df['dia_semana'] == 6)] += (df['Duration'][(df['Duration'] >= timedelta(seconds=1, minutes=45)) & (df['dia_semana'] == 6)] - ajuste).dt.total_seconds() // 1800 * 39

<ipython-input-6-a506c24c5507>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Cargo_adicional'][(df['Duration'] >= timedelta(seconds=1, minutes=30)) & (df['dia_semana'] != 6)] = 28
<ipython-input-6-a506c24c5507>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Cargo_adicional'][(df['Duration'] >= timedelta(seconds=1, minutes=60)) & (df['dia_semana'] != 6)] += (df['Duration'][(df['Duration'] >= timedelta(seconds=1, minutes=30)) & (df['dia_semana'] != 6)] - ajuste).dt.total_seconds() // 1800 * 39
<ipython-input-6-a506c24c5507>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

*   Analizar el precio promedio de usuario por año, tomando en cuanto los cargos adicionales.
*   Calcular la velocidad promedio utilizada
*   Comparación con los costos y tiempo de TP en Querétaro

In [ ]:
#Precio promedio de usuario por año
#Es razonable tomar solamente los viajes que no duren más de 3 horas
def promedio_anual_function(group):
  return((sum(group['Cargo_adicional']) + 478 * group['User_Id'].nunique()) / group['User_Id'].nunique())
Analisis_anual = df[df['Duration'] < timedelta(hours=3)].groupby('Year').apply(promedio_anual_function).reset_index()
Analisis_anual.columns = ['Year','Precio_Promedio_Por_Persona']

#Numero de Usuarios
def nom_usuarios_function(group):
  return(group['User_Id'].nunique())
Analisis_anual['Num_Users'] = df.groupby('Year').apply(nom_usuarios_function).reset_index()[0]

In [ ]:
Analisis_anual['Ingresos'] = Analisis_anual['Precio_Promedio_Por_Persona'] * Analisis_anual['Num_Users']
Numero_Estaciones = df.groupby('Year').Origin_Id.nunique().reset_index()
Numero_Estaciones.columns = ['Year','Num_Estaciones']
Analisis_anual = pd.merge(Analisis_anual,
         Numero_Estaciones,
         on='Year',
         how = 'left')

In [ ]:
Numero_Viajes =  df.groupby('Year').Trip_Id.count().reset_index()
Numero_Viajes.columns = ['Year','Num_viajes']
Analisis_anual = pd.merge(Analisis_anual,
         Numero_Viajes,
         on='Year',
         how = 'left')

In [ ]:
Analisis_anual['Active_Year'] = Analisis_anual['Year'] - 2013

In [ ]:
fig = px.bar(Analisis_anual, x="Year", y="Ingresos")
fig.update_traces(marker_color='green')
fig.update_layout(
    yaxis_title = 'Pesos $',
    xaxis_title = 'Año',
    title_text = 'Ingresos anuales MiBici',
    template = 'presentation'
)
fig.show()

## Calculo de velocidad promedio en Bicicleta

In [ ]:

def haversine_distance(lat1, lon1, lat2, lon2):
    # Convertir grados a radianes
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    # Radio de la Tierra en kilómetros
    R = 6371.0

    # Diferencias de coordenadas
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Fórmula del Haversine
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Distancia
    distancia = R * c
    return distancia

In [ ]:
Analisis_geografico = df[['Origin_Id','Destination_Id']].drop_duplicates()
Analisis_geografico = pd.merge(Analisis_geografico,
         nomen[['id','latitude','longitude']].rename(columns={'id': 'Origin_Id', 'latitude': 'Origin_latitude', 'longitude': 'Origin_longitude'}),
         on='Origin_Id',
         how = 'left')

Analisis_geografico = pd.merge(Analisis_geografico,
         nomen[['id','latitude','longitude']].rename(columns={'id': 'Destination_Id', 'latitude': 'Destination_latitude', 'longitude': 'Destination_longitude'}),
         on='Destination_Id',
         how = 'left')

df['Distancia'] = 0
def aplicar_distancia(row):
  return(haversine_distance(row['Origin_latitude'],row['Origin_longitude'],row['Destination_latitude'],row['Destination_longitude']))

Analisis_geografico['Distancia'] = Analisis_geografico.apply(aplicar_distancia, axis = 1)

Analisis_geografico

,Origin_Id,Destination_Id,Origin_latitude,Origin_longitude,Destination_latitude,Destination_longitude,Distancia
0,24,86,20.679389,-103.359130,20.669306,-103.368312,1.472932
1,48,279,20.675014,-103.365025,20.662310,-103.360110,1.502322
2,273,383,20.694060,-103.348240,20.689988,-103.344978,0.565776
3,79,60,20.675631,-103.361340,20.672641,-103.353542,0.876744
4,160,354,20.693110,-103.372980,20.688755,-103.345533,2.895929
...,...,...,...,...,...,...,...
82953,33,97,20.685652,-103.368215,20.726196,-103.392672,5.176461
82954,116,101,20.686920,-103.334670,20.726558,-103.389242,7.186449
82955,116,104,20.686920,-103.334670,20.735274,-103.399746,8.644193
82956,94,107,20.699950,-103.342347,20.723740,-103.388657,5.495245


In [ ]:
tiempo_mediano_trayecto = df.groupby(["Origin_Id","Destination_Id"]).Duration.median().reset_index()
tiempo_mediano_trayecto['Duration'] = tiempo_mediano_trayecto['Duration'].apply(lambda x: x.total_seconds()) / 3600
tiempo_mediano_trayecto

,Origin_Id,Destination_Id,Duration
0,2,2,0.176111
1,2,3,0.183333
2,2,4,0.212778
3,2,5,0.303750
4,2,6,0.234583
...,...,...,...
82953,388,374,0.332222
82954,388,376,0.158056
82955,388,383,0.343056
82956,388,386,0.213889


In [ ]:
Analisis_geografico = pd.merge(Analisis_geografico,
                               tiempo_mediano_trayecto,
                               on = ["Origin_Id","Destination_Id"],
                               how = 'left')
Analisis_geografico

,Origin_Id,Destination_Id,Origin_latitude,Origin_longitude,Destination_latitude,Destination_longitude,Distancia,Duration
0,24,86,20.679389,-103.359130,20.669306,-103.368312,1.472932,0.194861
1,48,279,20.675014,-103.365025,20.662310,-103.360110,1.502322,0.195694
2,273,383,20.694060,-103.348240,20.689988,-103.344978,0.565776,0.079167
3,79,60,20.675631,-103.361340,20.672641,-103.353542,0.876744,0.121667
4,160,354,20.693110,-103.372980,20.688755,-103.345533,2.895929,0.255000
...,...,...,...,...,...,...,...,...
82953,33,97,20.685652,-103.368215,20.726196,-103.392672,5.176461,0.801111
82954,116,101,20.686920,-103.334670,20.726558,-103.389242,7.186449,0.306528
82955,116,104,20.686920,-103.334670,20.735274,-103.399746,8.644193,0.171667
82956,94,107,20.699950,-103.342347,20.723740,-103.388657,5.495245,0.101944


In [ ]:
Analisis_geografico['Velocidad'] = Analisis_geografico.Distancia/Analisis_geografico.Duration

In [ ]:
#Velocidad promedio
print("Velocidad promedio",Analisis_geografico['Velocidad'].mean())

Velocidad promedio 10.113526496389726


In [ ]:
Analisis_geografico

,Origin_Id,Destination_Id,Origin_latitude,Origin_longitude,Destination_latitude,Destination_longitude,Distancia,Duration,Velocidad
0,24,86,20.679389,-103.359130,20.669306,-103.368312,1.472932,0.194861,7.558881
1,48,279,20.675014,-103.365025,20.662310,-103.360110,1.502322,0.195694,7.676878
2,273,383,20.694060,-103.348240,20.689988,-103.344978,0.565776,0.079167,7.146647
3,79,60,20.675631,-103.361340,20.672641,-103.353542,0.876744,0.121667,7.206119
4,160,354,20.693110,-103.372980,20.688755,-103.345533,2.895929,0.255000,11.356584
...,...,...,...,...,...,...,...,...,...
82953,33,97,20.685652,-103.368215,20.726196,-103.392672,5.176461,0.801111,6.461602
82954,116,101,20.686920,-103.334670,20.726558,-103.389242,7.186449,0.306528,23.444691
82955,116,104,20.686920,-103.334670,20.735274,-103.399746,8.644193,0.171667,50.354523
82956,94,107,20.699950,-103.342347,20.723740,-103.388657,5.495245,0.101944,53.904310


from matplotlib import pyplot as plt
Analisis_geografico['Origin_Id'].plot(kind='hist', bins=20, title='Origin_Id')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
Analisis_geografico['Destination_Id'].plot(kind='hist', bins=20, title='Destination_Id')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
Analisis_geografico['Origin_latitude'].plot(kind='hist', bins=20, title='Origin_latitude')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
Analisis_geografico['Origin_longitude'].plot(kind='hist', bins=20, title='Origin_longitude')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
Analisis_geografico.plot(kind='scatter', x='Origin_longitude', y='Destination_latitude', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
Analisis_geografico['Origin_Id'].plot(kind='line', figsize=(8, 4), title='Origin_Id')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
Analisis_geografico['Destination_Id'].plot(kind='line', figsize=(8, 4), title='Destination_Id')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
Analisis_geografico['Origin_latitude'].plot(kind='line', figsize=(8, 4), title='Origin_latitude')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
Analisis_geografico['Origin_longitude'].plot(kind='line', figsize=(8, 4), title='Origin_longitude')
plt.gca().spines[['top', 'right']].set_visible(False)

# Numero de estaciones por año

In [ ]:

Numero_Estaciones

,Year,Num_Estaciones
0,2014,86
1,2015,116
2,2016,240
3,2017,239
4,2018,277
5,2019,275
6,2020,276
7,2021,300
8,2022,300
9,2023,359


In [ ]:
fig = px.bar(Analisis_anual, x = "Year", y = "Num_Estaciones")
fig.update_traces(marker_color='green')
fig.update_layout(
    yaxis_title = 'Estaciones',
    xaxis_title = 'Año',
    title_text = 'Número de estaciones por año',
    template = 'presentation'
)
fig.show()

In [ ]:
fig = px.bar(Analisis_anual, x = "Year", y = "Num_viajes")
fig.update_traces(marker_color='green')
fig.update_layout(
    yaxis_title = 'Viajes',
    xaxis_title = 'Año',
    title_text = 'Número de Viajes por año',
    template = 'presentation'
)
fig.show()

In [ ]:
Analisis_anual['Autos_menos'] = Analisis_anual['Num_viajes']/1.6

In [ ]:
fig = px.bar(Analisis_anual, x = "Year", y = "Autos_menos")
fig.update_traces(marker_color='green')
fig.update_layout(
    yaxis_title = 'Autos menos',
    xaxis_title = 'Año',
    title_text = 'Número de Viajes en auto ahorrados',
    template = 'presentation'
)
fig.show()

In [ ]:
Analisis_anual

,Year,Precio_Promedio_Por_Persona,Num_Users,Ingresos,Num_Estaciones,Num_viajes,Active_Year
0,2014,485.507148,1469,7.132100e+05,86,21856,1
1,2015,510.784944,4172,2.130995e+06,116,449066,2
2,2016,519.687508,8286,4.306131e+06,240,907986,3
3,2017,538.183986,16864,9.075935e+06,239,2446270,4
4,2018,534.304926,23106,1.234565e+07,277,3307306,5
5,2019,540.387873,28316,1.530162e+07,275,4497615,6
6,2020,526.955466,23225,1.223854e+07,276,2774019,7
7,2021,529.770112,19990,1.059010e+07,300,3063689,8
8,2022,528.483043,24627,1.301495e+07,300,3984646,9
9,2023,526.521992,26611,1.401128e+07,359,4036822,10


In [ ]:
columns_to_choose = Analisis_anual.columns
columns_to_choose.drop('Ingresos')

Index(['Year', 'Precio_Promedio_Por_Persona', 'Num_Users', 'Num_Estaciones',
       'Num_viajes', 'Active_Year'],
      dtype='object')

In [ ]:
fig = px.scatter_matrix(Analisis_anual,
                        dimensions = Analisis_anual[['Active_Year', 'Precio_Promedio_Por_Persona', 'Num_Users', 'Num_Estaciones']],
                        title = 'Pares de Variables',
                        height = 1000,
                        color  = 'Ingresos')
fig.show()

# Creación del modelo

In [ ]:
from sklearn.model_selection import train_test_split #Separar el modelo
from sklearn.preprocessing import StandardScaler #Normalizar datos
from sklearn.svm import SVR #Maquinas de vectores
from sklearn.tree import DecisionTreeRegressor, plot_tree #Arboles de decision
from sklearn.ensemble import RandomForestRegressor #Arboles aleatorios
from sklearn.metrics import mean_squared_error, r2_score #Formas de medir el modelo
from sklearn.linear_model import LinearRegression

In [ ]:
#Creamos el modelo
X = Analisis_anual[['Active_Year','Num_Estaciones']]
y = Analisis_anual['Ingresos']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 26)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [ ]:
svm = SVR()
svm.fit(X_train_scaled,y_train)

In [ ]:
y_pred_svm = svm.predict(X_test_scaled)
mse_svm = mean_squared_error(y_test, y_pred = y_pred_svm)
print(mse_svm) #Entre más cercano sea r^2 a 0, mejor va a ser el modelo

In [ ]:
X_test

In [ ]:
X_train_scaled

In [ ]:
Analisis_anual

# Arboles

In [ ]:
dt = DecisionTreeRegressor(random_state = 26)
dt.fit(X_train_scaled, y_train)

In [ ]:
y_pred_dt = dt.predict(X_test_scaled)
mse_dt = mean_squared_error(y_test, y_pred_dt)
print(mse_dt)

#Métdo de clasificacion
El método no funcionó entonces intentemos uno de clasificición

In [ ]:
import numpy as np
from scipy.cluster.hierarchy import linkage, fcluster
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import pdist
import plotly.graph_objects as go

In [ ]:
def haversine(coord1, coord2):
    """
    Calcula la distancia Haversine entre dos puntos
    en coordenadas (latitud, longitud).
    """
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    R = 6371.0  # Radio de la Tierra en kilómetros

    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = np.sin(dlat / 2)**2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

In [ ]:
coordinates = nomen[nomen['status'] == 'IN_SERVICE'][['latitude','longitude']]

In [ ]:
dist_matrix = pdist(coordinates, metric = haversine)
dist_matrix

array([1.78989385, 2.02694716, 3.16220693, ..., 3.01612495, 1.60458154,
       1.41266324])

In [ ]:
# Clustering jerárquico
Z = linkage(dist_matrix, method='complete')

# Obtener clusters con un umbral de distancia de 5 km
max_distance = 5  # kilómetros
clusters = fcluster(Z, t=max_distance, criterion='distance')

In [ ]:
data = np.column_stack((coordinates, clusters))

# Convertir a un DataFrame para facilitar la manipulación
data = pd.DataFrame(data, columns=['Latitude', 'Longitude', 'Cluster'])
data['Cluster'] = data['Cluster'].astype(int)

In [ ]:
data.Cluster

0      6
1      6
2      6
3      5
4      4
      ..
354    4
355    4
356    4
357    4
358    4
Name: Cluster, Length: 359, dtype: int64

In [ ]:
fig = px.scatter_mapbox(data, lat = "Latitude", lon = "Longitude", color = 'Cluster')
fig.update_layout(mapbox_style="carto-positron")
fig.update_traces(marker=dict(size=15))
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

#INTENTO CON QROBICI

In [ ]:
qrobici = pd.read_csv("Qrobici.csv")

In [ ]:
coordinates = qrobici[['Latitude','Longitude']]
dist_matrix = pdist(coordinates, metric = haversine)

In [ ]:
# Clustering jerárquico
Z = linkage(dist_matrix, method='complete')

# Obtener clusters con un umbral de distancia de 50 km
max_distance = 5  # kilómetros
clusters = fcluster(Z, t=max_distance, criterion='distance')

In [ ]:
data = np.column_stack((coordinates, clusters))

# Convertir a un DataFrame para facilitar la manipulación
data = pd.DataFrame(data, columns=['Latitude', 'Longitude', 'Cluster'])
data['Cluster'] = data['Cluster'].astype(int)

In [ ]:
fig = px.scatter_mapbox(data, lat = "Latitude", lon = "Longitude", color = 'Cluster')
fig.update_layout(mapbox_style = 'carto-positron')
fig.update_traces(marker=dict(size=15, color='blue'))
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

# Extra

In [ ]:
fig = px.line(Analisis_anual, x="Year", y="Num_Users")
fig.update_traces(line=dict(color='green'))
fig.update_layout(
    yaxis_title = 'Número de usuarios',
    xaxis_title = 'Año',
    title_text = 'Número de usuarios por año',
    template = 'presentation'
)
fig.show()

In [ ]:
Analisis_anual

In [ ]:
coefficients = np.polyfit(Analisis_anual['Active_Year'][:-1], Analisis_anual['Num_Users'][:-1], 1)
print("Tendencia de usuarios ",coefficients[0])

In [ ]:
Analisis_anual

,Year,Precio_Promedio_Por_Persona,Num_Users,Ingresos,Num_Estaciones,Num_viajes,Active_Year,Autos_menos
0,2014,485.507148,1469,7.132100e+05,86,21856,1,13660.000
1,2015,510.784944,4172,2.130995e+06,116,449066,2,280666.250
2,2016,519.687508,8286,4.306131e+06,240,907986,3,567491.250
3,2017,538.183986,16864,9.075935e+06,239,2446270,4,1528918.750
4,2018,534.304926,23106,1.234565e+07,277,3307306,5,2067066.250
5,2019,540.387873,28316,1.530162e+07,275,4497615,6,2811009.375
6,2020,526.955466,23225,1.223854e+07,276,2774019,7,1733761.875
7,2021,529.770112,19990,1.059010e+07,300,3063689,8,1914805.625
8,2022,528.483043,24627,1.301495e+07,300,3984646,9,2490403.750
9,2023,526.521992,26611,1.401128e+07,359,4036822,10,2523013.750


In [ ]:
Analisis_geografico

In [ ]:
Analisis_geografico.Velocidad.mean()